Deep Learning
=============



# Assignment 3

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic with L2 regularization 

在`4.Optimizer`之前添加`regularization` 项到`loss`中.

In [5]:
batch_size = 128
learning_rate = 0.5 
beta = 0.05

graph = tf.Graph()
with graph.as_default():

  # 1.Input data. 
  # For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 2.Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels])) # (784,10)
  biases = tf.Variable(tf.zeros([num_labels])) #(10, )
  
  # 3.Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Add the regularization term to the loss.
  loss += beta * tf.nn.l2_loss(weights)
  
  # 4.Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # 5.Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



这一部分与`2_fullyconnected.ipynb`中一样

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print('Minibatch loss at step {}: {:.3f}. Minibatch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(), valid_labels)))
      
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 165.491. Minibatch acc: 14.1%, Valid acc: 17.6%.
Minibatch loss at step 500: 1.045. Minibatch acc: 78.1%, Valid acc: 80.2%.
Minibatch loss at step 1000: 0.858. Minibatch acc: 82.8%, Valid acc: 79.8%.
Minibatch loss at step 1500: 0.952. Minibatch acc: 81.2%, Valid acc: 79.3%.
Minibatch loss at step 2000: 0.991. Minibatch acc: 78.9%, Valid acc: 79.4%.
Minibatch loss at step 2500: 0.868. Minibatch acc: 80.5%, Valid acc: 79.3%.
Minibatch loss at step 3000: 0.836. Minibatch acc: 83.6%, Valid acc: 79.0%.
Test accuracy: 86.0%


### Neural NetworkL2 regularization 

In [9]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5 
beta = 0.005

graph = tf.Graph()

with graph.as_default():

  # 1.Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 2.Variables.
  # 隐藏层权重和偏差
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes])) # (784,1024)
  biases_1 = tf.Variable(tf.zeros([hidden_nodes])) # (1024, )
  # 输出层权重和偏差
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels])) # (1024,10)
  biases_2 = tf.Variable(tf.zeros([num_labels])) # (10，)
  
  # 3.Training computation.
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    return tf.matmul(h1, weights_2) + biases_2
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
  
  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

  # 4.Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # 5.Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        print('Minibatch loss at step {}: {:.3f}. batch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1954.975. batch acc: 10.9%, Valid acc: 30.1%.
Minibatch loss at step 500: 127.365. batch acc: 82.8%, Valid acc: 82.6%.
Minibatch loss at step 1000: 10.899. batch acc: 88.3%, Valid acc: 85.4%.
Minibatch loss at step 1500: 1.493. batch acc: 85.2%, Valid acc: 84.9%.
Minibatch loss at step 2000: 0.713. batch acc: 84.4%, Valid acc: 85.1%.
Minibatch loss at step 2500: 0.638. batch acc: 81.2%, Valid acc: 85.5%.
Minibatch loss at step 3000: 0.636. batch acc: 85.2%, Valid acc: 85.1%.
Test accuracy: 91.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

这一步我一直在思考：`Restrict your training data to just a few batches`是什么意思，最开始是把batch_size 设置的很大，结果效果要比前面的都好，然后又把训练步数减少，测试集的准确率依然很高，也不是过拟合

把batch_size 设置的大一点

In [11]:
batch_size = 2048
hidden_nodes = 1024
learning_rate = 0.5 
beta = 0.005

graph = tf.Graph()

with graph.as_default():

  # 1.Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 2.Variables.
  # 隐藏层权重和偏差
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes])) # (784,1024)
  biases_1 = tf.Variable(tf.zeros([hidden_nodes])) # (1024, )
  # 输出层权重和偏差
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels])) # (1024,10)
  biases_2 = tf.Variable(tf.zeros([num_labels])) # (10，)
  
  # 3.Training computation.
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    return tf.matmul(h1, weights_2) + biases_2
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
  
  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

  # 4.Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # 5.Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        print('Minibatch loss at step {}: {:.3f}. batch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1949.589. batch acc: 15.5%, Valid acc: 33.8%.
Minibatch loss at step 500: 126.444. batch acc: 85.4%, Valid acc: 84.8%.
Minibatch loss at step 1000: 10.870. batch acc: 87.5%, Valid acc: 87.3%.
Minibatch loss at step 1500: 1.453. batch acc: 87.6%, Valid acc: 86.6%.
Minibatch loss at step 2000: 0.668. batch acc: 87.4%, Valid acc: 86.7%.
Minibatch loss at step 2500: 0.584. batch acc: 86.9%, Valid acc: 86.8%.
Minibatch loss at step 3000: 0.593. batch acc: 87.0%, Valid acc: 87.0%.
Test accuracy: 93.0%


把训练步数减少

In [14]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5 
beta = 0.005

graph = tf.Graph()

with graph.as_default():

  # 1.Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 2.Variables.
  # 隐藏层权重和偏差
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes])) # (784,1024)
  biases_1 = tf.Variable(tf.zeros([hidden_nodes])) # (1024, )
  # 输出层权重和偏差
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels])) # (1024,10)
  biases_2 = tf.Variable(tf.zeros([num_labels])) # (10，)
  
  # 3.Training computation.
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    return tf.matmul(h1, weights_2) + biases_2
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
  
  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

  # 4.Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # 5.Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [15]:
num_steps = 301

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
        print('Minibatch loss at step {}: {:.3f}. batch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1915.114. batch acc: 9.4%, Valid acc: 27.8%.
Minibatch loss at step 50: 1278.033. batch acc: 69.5%, Valid acc: 73.5%.
Minibatch loss at step 100: 959.762. batch acc: 77.3%, Valid acc: 78.1%.
Minibatch loss at step 150: 745.796. batch acc: 76.6%, Valid acc: 79.7%.
Minibatch loss at step 200: 584.136. batch acc: 75.8%, Valid acc: 78.6%.
Minibatch loss at step 250: 451.124. batch acc: 81.2%, Valid acc: 78.6%.
Minibatch loss at step 300: 347.583. batch acc: 78.1%, Valid acc: 78.3%.
Test accuracy: 85.1%


把训练数据减少

In [17]:
train_dataset_restricted = train_dataset[:3000, :]
train_labels_restricted = train_labels[:3000, :]

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_restricted.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_restricted[offset:(offset + batch_size), :]
    batch_labels = train_labels_restricted[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        print('Minibatch loss at step {}: {:.3f}. batch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1908.984. batch acc: 10.9%, Valid acc: 40.7%.
Minibatch loss at step 500: 128.552. batch acc: 98.4%, Valid acc: 80.3%.
Minibatch loss at step 1000: 10.719. batch acc: 100.0%, Valid acc: 82.9%.
Minibatch loss at step 1500: 1.287. batch acc: 98.4%, Valid acc: 82.7%.
Minibatch loss at step 2000: 0.470. batch acc: 100.0%, Valid acc: 83.1%.
Minibatch loss at step 2500: 0.406. batch acc: 99.2%, Valid acc: 82.1%.
Minibatch loss at step 3000: 0.355. batch acc: 100.0%, Valid acc: 81.8%.
Test accuracy: 88.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [22]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5 
beta = 0.005

graph = tf.Graph()

with graph.as_default():

  # 1.Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Placeholder to control dropout probability.
  keep_prob = tf.placeholder(tf.float32)

  # 2.Variables.
  # 隐藏层权重和偏差
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes])) # (784,1024)
  biases_1 = tf.Variable(tf.zeros([hidden_nodes])) # (1024, )
  # 输出层权重和偏差
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels])) # (1024,10)
  biases_2 = tf.Variable(tf.zeros([num_labels])) # (10，)
    
  # Training computation.
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    
    # Add dropout to the hidden layer.
    drop = tf.nn.dropout(h1, keep_prob)
    
    return tf.matmul(drop, weights_2) + biases_2
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))

  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [29]:
train_dataset_restricted = train_dataset[:3000, :]
train_labels_restricted = train_labels[:3000, :]

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    feed_dict_w_drop = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict_w_drop)
    if (step % 500 == 0):
        print('Minibatch loss at step {}: {:.3f}. batch acc: {:.1f}%, Valid acc: {:.1f}%.'\
                  .format(step, l,
                          accuracy(predictions, batch_labels),
                          accuracy(valid_prediction.eval(feed_dict=feed_dict), valid_labels)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 2081.080. batch acc: 10.9%, Valid acc: 23.6%.
Minibatch loss at step 500: 129.846. batch acc: 78.1%, Valid acc: 82.8%.
Minibatch loss at step 1000: 11.031. batch acc: 84.4%, Valid acc: 84.5%.
Minibatch loss at step 1500: 1.578. batch acc: 84.4%, Valid acc: 84.5%.
Minibatch loss at step 2000: 0.802. batch acc: 83.6%, Valid acc: 84.7%.
Minibatch loss at step 2500: 0.789. batch acc: 78.9%, Valid acc: 84.8%.
Minibatch loss at step 3000: 0.723. batch acc: 83.6%, Valid acc: 84.7%.
Test accuracy: 91.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
